# DeepSeek-Chat Görsel Sonuçlar

In [12]:
import os, json
import pandas as pd
import matplotlib.pyplot as plt

# 1) Tüm dosyaları topla
ratings_dir = "../data/ratings/deepseek_ratings"
rows = []
for fn in os.listdir(ratings_dir):
    if fn.endswith(".json"):
        with open(os.path.join(ratings_dir, fn), "r", encoding="utf-8") as f:
            rows.extend(json.load(f))

df = pd.DataFrame(rows)

# 2) ratings kolonunu aç ve overall ekle
rat = df["ratings"].apply(pd.Series)
df = pd.concat([df.drop(columns=["ratings"]), rat], axis=1)
df["overall"] = df[["word_usage", "clarity", "grammar", "naturalness"]].mean(axis=1)

# 3) Model×Seviye ortalamaları
model_level_avg = (
    df.groupby(["model", "level"], as_index=False)[
        ["word_usage", "clarity", "grammar", "naturalness", "overall"]
    ].mean()
)

# 4) Çıktı klasörü
out_dir = "../data/ratings/deepseek_ratings/plots"
os.makedirs(out_dir, exist_ok=True)

# 5) Overall grafiği
pivot = model_level_avg.pivot(index="model", columns="level", values="overall")
pivot.plot(kind="bar", figsize=(12, 6))
plt.ylabel("Average Overall Score")
plt.title("Model Performance by CEFR Level")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig(os.path.join(out_dir, "overall_performance.png"), dpi=300)
plt.close()

# 6) Kriter bazlı grafikler
for col in ["word_usage", "clarity", "grammar", "naturalness"]:
    pivot_k = model_level_avg.pivot(index="model", columns="level", values=col)
    pivot_k.plot(kind="bar", figsize=(12, 6))
    plt.ylabel(f"Average {col.capitalize()}")
    plt.title(f"Model × Level — {col.capitalize()}")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, f"{col}_performance.png"), dpi=300)
    plt.close()

print(f"✅ Tüm grafikler kaydedildi: {out_dir}")



✅ Tüm grafikler kaydedildi: ../data/ratings/deepseek_ratings/plots


# DeepSeek-Chat CSV Sonuçları

In [ ]:
import os, json
import pandas as pd

# --- 1) Verileri yükle ---
ratings_dir = "../data/ratings/deepseek_ratings"
rows = []
for fn in os.listdir(ratings_dir):
    if fn.endswith(".json"):
        with open(os.path.join(ratings_dir, fn), "r", encoding="utf-8") as f:
            rows.extend(json.load(f))

df = pd.DataFrame(rows)

# --- 2) ratings dict'ini aç ---
rat = df["ratings"].apply(pd.Series)
df = pd.concat([df.drop(columns=["ratings"]), rat], axis=1)

# overall ekle
df["overall"] = df[["word_usage", "clarity", "grammar", "naturalness"]].mean(axis=1)

# --- 3) Model × Seviye ortalamaları ---
model_level_avg = (
    df.groupby(["model", "level"], as_index=False)[
        ["word_usage", "clarity", "grammar", "naturalness", "overall"]
    ].mean()
)

# --- 4) Her kriter için genel sıralama (tüm seviyeler birleşik) ---
criteria_ranking = (
    df.groupby("model")[["word_usage", "clarity", "grammar", "naturalness", "overall"]]
    .mean()
    .sort_values("overall", ascending=False)
    .reset_index()
)

# --- 5) Genel sıralama sadece overall üzerinden ---
overall_ranking = (
    df.groupby("model")["overall"]
    .mean()
    .sort_values(ascending=False)
    .reset_index()
)

# --- 6) CSV olarak kaydet ---
out_dir = "../data/ratings/deepseek_ratings/analysis_results"
os.makedirs(out_dir, exist_ok=True)

model_level_avg.to_csv(os.path.join(out_dir, "model_level_avg.csv"), index=False)
criteria_ranking.to_csv(os.path.join(out_dir, "criteria_ranking.csv"), index=False)
overall_ranking.to_csv(os.path.join(out_dir, "overall_ranking.csv"), index=False)

print("✅ Analiz CSV dosyaları kaydedildi:")
print(f"- {os.path.join(out_dir, 'model_level_avg.csv')}")
print(f"- {os.path.join(out_dir, 'criteria_ranking.csv')}")
print(f"- {os.path.join(out_dir, 'overall_ranking.csv')}")


✅ Analiz CSV dosyaları kaydedildi:
- ../data/analysis_results/model_level_avg.csv
- ../data/analysis_results/criteria_ranking.csv
- ../data/analysis_results/overall_ranking.csv


# ChatGPT-5 Sonuçlarının Analizi

In [16]:
import os, json
import pandas as pd

# --- 1) Verileri yükle ---
ratings_dir = "../data/ratings/chatgpt_ratings"
rows = []
for fn in os.listdir(ratings_dir):
    if fn.endswith(".json"):
        with open(os.path.join(ratings_dir, fn), "r", encoding="utf-8") as f:
            rows.extend(json.load(f))

df = pd.DataFrame(rows)

# --- 2) ratings dict'ini aç ---
rat = df["ratings"].apply(pd.Series)
df = pd.concat([df.drop(columns=["ratings"]), rat], axis=1)

# overall ekle
df["overall"] = df[["word_usage", "clarity", "grammar", "naturalness"]].mean(axis=1)

# --- 3) Model × Seviye ortalamaları ---
model_level_avg = (
    df.groupby(["model", "level"], as_index=False)[
        ["word_usage", "clarity", "grammar", "naturalness", "overall"]
    ].mean()
)

# --- 4) Her kriter için genel sıralama (tüm seviyeler birleşik) ---
criteria_ranking = (
    df.groupby("model")[["word_usage", "clarity", "grammar", "naturalness", "overall"]]
    .mean()
    .sort_values("overall", ascending=False)
    .reset_index()
)

# --- 5) Genel sıralama sadece overall üzerinden ---
overall_ranking = (
    df.groupby("model")["overall"]
    .mean()
    .sort_values(ascending=False)
    .reset_index()
)

# --- 6) CSV olarak kaydet ---
out_dir = "../data/ratings/chatgpt_ratings/analysis_results"
os.makedirs(out_dir, exist_ok=True)

model_level_avg.to_csv(os.path.join(out_dir, "model_level_avg.csv"), index=False)
criteria_ranking.to_csv(os.path.join(out_dir, "criteria_ranking.csv"), index=False)
overall_ranking.to_csv(os.path.join(out_dir, "overall_ranking.csv"), index=False)

print("✅ Analiz CSV dosyaları kaydedildi:")
print(f"- {os.path.join(out_dir, 'model_level_avg.csv')}")
print(f"- {os.path.join(out_dir, 'criteria_ranking.csv')}")
print(f"- {os.path.join(out_dir, 'overall_ranking.csv')}")

✅ Analiz CSV dosyaları kaydedildi:
- ../data/ratings/chatgpt_ratings/analysis_results/model_level_avg.csv
- ../data/ratings/chatgpt_ratings/analysis_results/criteria_ranking.csv
- ../data/ratings/chatgpt_ratings/analysis_results/overall_ranking.csv


# ChapGPT-5 Görsel Sonuçlar

In [17]:
import os, json
import pandas as pd
import matplotlib.pyplot as plt

# 1) Tüm dosyaları topla
ratings_dir = "../data/ratings/chatgpt_ratings"
rows = []
for fn in os.listdir(ratings_dir):
    if fn.endswith(".json"):
        with open(os.path.join(ratings_dir, fn), "r", encoding="utf-8") as f:
            rows.extend(json.load(f))

df = pd.DataFrame(rows)

# 2) ratings kolonunu aç ve overall ekle
rat = df["ratings"].apply(pd.Series)
df = pd.concat([df.drop(columns=["ratings"]), rat], axis=1)
df["overall"] = df[["word_usage", "clarity", "grammar", "naturalness"]].mean(axis=1)

# 3) Model×Seviye ortalamaları
model_level_avg = (
    df.groupby(["model", "level"], as_index=False)[
        ["word_usage", "clarity", "grammar", "naturalness", "overall"]
    ].mean()
)

# 4) Çıktı klasörü
out_dir = "../data/ratings/chatgpt_ratings/plots"
os.makedirs(out_dir, exist_ok=True)

# 5) Overall grafiği
pivot = model_level_avg.pivot(index="model", columns="level", values="overall")
pivot.plot(kind="bar", figsize=(12, 6))
plt.ylabel("Average Overall Score")
plt.title("Model Performance by CEFR Level")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig(os.path.join(out_dir, "overall_performance.png"), dpi=300)
plt.close()

# 6) Kriter bazlı grafikler
for col in ["word_usage", "clarity", "grammar", "naturalness"]:
    pivot_k = model_level_avg.pivot(index="model", columns="level", values=col)
    pivot_k.plot(kind="bar", figsize=(12, 6))
    plt.ylabel(f"Average {col.capitalize()}")
    plt.title(f"Model × Level — {col.capitalize()}")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, f"{col}_performance.png"), dpi=300)
    plt.close()

print(f"✅ Tüm grafikler kaydedildi: {out_dir}")

✅ Tüm grafikler kaydedildi: ../data/ratings/chatgpt_ratings/plots
